In [1]:
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
duckdb.sql(
    """SELECT *
           FROM read_json_auto('Data/*/*.json') LIMIT 1"""
           )



┌─────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [4]:
# Define Function to add move numbers to pgn
def add_move_numbers(pgn: list) -> str:
    # Remove any existing move numbers and split into individual moves
    moves = pgn
    
    # Reconstruct the PGN with move numbers
    formatted_pgn = []
    move_number = 1
    for i in range(0, len(pgn), 2):
        # Add the move number and the two moves (white and black)
        formatted_pgn.append(f"{move_number}. {moves[i]} {moves[i+1] if i+1 < len(moves) else ''}")
        move_number += 1
    
    # Join the formatted moves into a single string
    return ' '.join(formatted_pgn)

# duckdb.remove_function('add_move_numbers')
duckdb.create_function('add_move_numbers', add_move_numbers)


def get_opening_family(opening_name: str) -> str:
    # Get the parent name of the move numbers

    if ":" in opening_name:
        name_splitted = opening_name.split(":")
        return name_splitted[0]

    else:
        return opening_name
    
duckdb.create_function('get_opening_family', get_opening_family)


/tmp/ipykernel_83981/3657449439.py:18: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  duckdb.create_function('add_move_numbers', add_move_numbers)


In [18]:
fact_table = duckdb.sql(
    """SELECT url as game_url,
                time_control as time_control,
                rated as rated,
                time_class as time_class,
                rules as rules,
                white.rating as white_rating,
                white.result as white_result,
                black.rating as black_rating,
                black.result as black_result,    
                REGEXP_EXTRACT(pgn, '\[Event "(.*?)"', 1) as pgn_event,
                REGEXP_EXTRACT(pgn, '\[Site "(.*?)"', 1) as pgn_site,
                REGEXP_EXTRACT(pgn, '\[Date "(.*?)"', 1) as pgn_date, 
                STRPTIME(REPLACE(REGEXP_EXTRACT(pgn, '\[Date "(.*?)"', 1), '.', '/'), '%Y/%m/%d') AS game_date,
                REGEXP_EXTRACT(pgn, '\[White "(.*?)"', 1) as pgn_white_user,   
                REGEXP_EXTRACT(pgn, '\[Black "(.*?)"', 1) as pgn_black_user,
                REGEXP_EXTRACT(pgn, '\[Result "(.*?)"', 1) as pgn_result,
                REGEXP_EXTRACT(pgn, '\[CurrentPosition "(.*?)"', 1) as pgn_current_position,
                REGEXP_EXTRACT(pgn, '\[Timezone "(.*?)"', 1) as pgn_timezone,
                REGEXP_EXTRACT(pgn, '\[ECO "(.*?)"', 1) as pgn_eco,
                REGEXP_EXTRACT(pgn, '\[ECOUrl "(.*?)"', 1) as pgn_eco_url,
                REGEXP_EXTRACT(pgn, '\[StartTime "(.*?)"', 1) as start_time,
                REGEXP_EXTRACT(pgn, '\[EndTime "(.*?)"', 1) as End_time,
                ARRAY_TO_STRING(REGEXP_EXTRACT_ALL(pgn, '\. (.*?) {\[', 1), ' ') as pgn_raw,
                add_move_numbers(REGEXP_EXTRACT_ALL(pgn, '\. (.*?) {\[', 1)) as pgn_trans
                                
            FROM read_json_auto('Data/*/*.json')"""
           )

fact_table

┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬─────────────────────┬─────────────────┬────────────────┬────────────┬───────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [14]:
duckdb.sql("""
           SELECT COUNT(pgn_raw) as number_of_games, pgn_eco, pgn_eco_url
           FROM fact_table
           GROUP BY pgn_eco, pgn_eco_url
           ORDER BY number_of_games DESC
           """)

┌─────────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│ number_of_games │ pgn_eco │                                          pgn_eco_url                                          │
│      int64      │ varchar │                                            varchar                                            │
├─────────────────┼─────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│              24 │ C00     │ https://www.chess.com/openings/French-Defense-Knight-Variation-2...d5-3.exd5-exd5             │
│              24 │ A40     │ https://www.chess.com/openings/Englund-Gambit-Hartlaub-Charlick-Gambit                        │
│              24 │ C01     │ https://www.chess.com/openings/French-Defense-Exchange-Variation-3...exd5-4.Nf3               │
│              22 │ C00     │ https://www.chess.com/openings/French-Defense                                           

In [15]:
# # Transform openings csv 
# openings = conn.sql("""
#             SELECT "eco-volume" as eco_family, eco, name, pgn
#             FROM 'hf://datasets/Lichess/chess-openings/data/train-00000-of-00001.parquet'
            
#          """
# ).df()
# openings

openings = duckdb.sql("""
    SELECT *, get_opening_family(name) as family_name
           FROM read_csv('openings.csv')
           
""")
openings

┌────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┐
│ eco_family │   eco   │                                      name                                       │                                                                      pgn                                                                       │      family_name      │
│  varchar   │ varchar │                                     varchar                                     │                                                                    varchar                                                                     │        varchar        │
├────────────┼─────────┼─────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────

In [10]:
duckdb.query("""SELECT *
                FROM fact_table as ft
                WHERE pgn_trans LIKE '%1. e4 e6%'         
             """)

┌──────────────────────────────────────────────┬──────────────┬─────────┬────────────┬─────────┬──────────────┬──────────────┬──────────────┬──────────────┬────────────┬───────────┬────────────┬─────────────────┬────────────────┬────────────┬─────────────────────────────────────────────────────────────┬──────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────┬────────────┬──────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────

In [11]:
duckdb.sql("""
            SELECT family_name, count(*) as num_opens FROM openings GROUP BY family_name ORDER BY num_opens desc;
        """)

┌─────────────────────────┬───────────┐
│       family_name       │ num_opens │
│         varchar         │   int64   │
├─────────────────────────┼───────────┤
│ Sicilian Defense        │       371 │
│ Ruy Lopez               │       229 │
│ French Defense          │       188 │
│ Italian Game            │       169 │
│ Queen's Gambit Declined │       163 │
│ English Opening         │       146 │
│ King's Gambit Accepted  │       137 │
│ King's Indian Defense   │       112 │
│ Caro-Kann Defense       │       103 │
│ Nimzo-Indian Defense    │        92 │
│        ·                │         · │
│        ·                │         · │
│        ·                │         · │
│ Gunderam Defense        │         1 │
│ Dresden Opening         │         1 │
│ Global Opening          │         1 │
│ Saragossa Opening       │         1 │
│ Valencia Opening        │         1 │
│ Australian Defense      │         1 │
│ Wade Defense            │         1 │
│ Paleface Attack         │         1 │


In [ ]:
# Create Dimension tables for Openings, Date, Player and GameType

# Openings Dimension Table
conn.execute("""
CREATE TABLE IF NOT EXISTS dim_openings AS
SELECT DISTINCT
    pgn_eco AS opening_code,
    pgn_eco_url AS opening_url
FROM read_json_auto('Data/*/*.json')
""")


In [ ]:

# Create Openings Dimension Tables.
conn.execute("""
        CREATE TABLE IF NOT EXISTS dim_openings (
             eco_family NOT NULL VARCHAR
             eco_code VARCHAR
             eco_name VARCHAR
             eco_pgn VARCHAR PRIMARY KEY UNIQUE
             eco_general_name VAR CHAR
             )
    """)

ParserException: Parser Error: syntax error at end of input